In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import randint

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.datasets import make_classification
from sklearn.preprocessing import binarize, LabelEncoder, MinMaxScaler

train_df = pd.read_csv('mental.csv')
print(train_df.shape)
print(train_df.describe())
print(train_df.info())

(1259, 29)
                Age
count  1.259000e+03
mean   7.942815e+07
std    2.818299e+09
min   -1.726000e+03
25%    2.700000e+01
50%    3.100000e+01
75%    3.600000e+01
max    1.000000e+11
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1259 entries, 0 to 1258
Data columns (total 29 columns):
Timestamp                                                                            1259 non-null object
Age                                                                                  1259 non-null int64
Gender                                                                               1259 non-null object
Country                                                                              1259 non-null object
state                                                                                744 non-null object
self_employed                                                                        1241 non-null object
family_history                                                      

In [2]:
train_df = train_df.drop(['Country'], axis= 1)
train_df = train_df.drop(['comments'], axis= 1)
train_df = train_df.drop(['state'], axis= 1)
train_df = train_df.drop(['Timestamp'], axis= 1)


train_df.isnull().sum().max()
train_df.head(5)

,Age,Gender,self_employed,family_history,treatment,work_interfere,no_employees,remote_work,tech_company,benefits,...,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,Have you had a mental health disorder in the past?,Have you been diagnosed with a mental health condition by a medical professional?
0,37,Female,NaN,No,Yes,Often,Jun-25,No,Yes,Yes,...,No,No,Some of them,Yes,No,Maybe,Yes,No,Yes,Yes
1,44,M,NaN,No,No,Rarely,More than 1000,No,No,Don't know,...,Maybe,No,No,No,No,No,Don't know,No,Yes,Yes
2,32,Male,NaN,No,No,Rarely,Jun-25,No,Yes,No,...,No,No,Yes,Yes,Yes,Yes,No,No,Maybe,No
3,31,Male,NaN,Yes,Yes,Often,26-100,No,Yes,No,...,Yes,Yes,Some of them,No,Maybe,Maybe,No,Yes,Yes,Yes
4,31,Male,NaN,No,No,Never,100-500,Yes,Yes,Yes,...,No,No,Some of them,Yes,Yes,Yes,Don't know,No,Yes,Yes


In [3]:
defaultInt = 0
defaultString = 'NaN'
defaultFloat = 0.0
intFeatures = ['Age']
stringFeatures = ['Gender', 'self_employed', 'family_history', 'treatment', 'work_interfere',
                 'no_employees', 'remote_work', 'tech_company', 'anonymity', 'leave', 'mental_health_consequence',
                 'phys_health_consequence', 'coworkers', 'supervisor', 'mental_health_interview', 'phys_health_interview',
                 'mental_vs_physical', 'obs_consequence', 'benefits', 'care_options', 'wellness_program',
                 'seek_help','Have you had a mental health disorder in the past?',
                  'Have you been diagnosed with a mental health condition by a medical professional?']
floatFeatures = []
for feature in train_df:
    if feature in intFeatures:
        train_df[feature] = train_df[feature].fillna(defaultInt)
    elif feature in stringFeatures:
        train_df[feature] = train_df[feature].fillna(defaultString)
    elif feature in floatFeatures:
        train_df[feature] = train_df[feature].fillna(defaultFloat)
    else:
        print('Error: Feature %s not recognized.' % feature)
train_df.head(5) 

,Age,Gender,self_employed,family_history,treatment,work_interfere,no_employees,remote_work,tech_company,benefits,...,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,Have you had a mental health disorder in the past?,Have you been diagnosed with a mental health condition by a medical professional?
0,37,Female,NaN,No,Yes,Often,Jun-25,No,Yes,Yes,...,No,No,Some of them,Yes,No,Maybe,Yes,No,Yes,Yes
1,44,M,NaN,No,No,Rarely,More than 1000,No,No,Don't know,...,Maybe,No,No,No,No,No,Don't know,No,Yes,Yes
2,32,Male,NaN,No,No,Rarely,Jun-25,No,Yes,No,...,No,No,Yes,Yes,Yes,Yes,No,No,Maybe,No
3,31,Male,NaN,Yes,Yes,Often,26-100,No,Yes,No,...,Yes,Yes,Some of them,No,Maybe,Maybe,No,Yes,Yes,Yes
4,31,Male,NaN,No,No,Never,100-500,Yes,Yes,Yes,...,No,No,Some of them,Yes,Yes,Yes,Don't know,No,Yes,Yes


In [4]:
gender = train_df['Gender'].str.lower()
gender = train_df['Gender'].unique()
male_str = ["male", "m", "male-ish", "maile", "mal", "male (cis)", "make", "male ", "man","msle", "mail", "malr","cis man", "Cis Male", "cis male"]
trans_str = ["trans-female", "something kinda male?", "queer/she/they", "non-binary","nah", "all", "enby", "fluid", "genderqueer", "androgyne", "agender", "male leaning androgynous", "guy (-ish) ^_^", "trans woman", "neuter", "female (trans)", "queer", "ostensibly male, unsure what that really means"]           
female_str = ["cis female", "f", "female", "woman",  "femake", "female ","cis-female/femme", "female (cis)", "femail"]
for (row, col) in train_df.iterrows():

    if str.lower(col.Gender) in male_str:
        train_df['Gender'].replace(to_replace=col.Gender, value='male', inplace=True)

    if str.lower(col.Gender) in female_str:
        train_df['Gender'].replace(to_replace=col.Gender, value='female', inplace=True)

    if str.lower(col.Gender) in trans_str:
        train_df['Gender'].replace(to_replace=col.Gender, value='trans', inplace=True)

stk_list = ['A little about you', 'p']
train_df = train_df[~train_df['Gender'].isin(stk_list)]

print(train_df['Gender'].unique())

['female' 'male' 'trans']


In [5]:
train_df['Age'].fillna(train_df['Age'].median(), inplace = True)
s = pd.Series(train_df['Age'])
s[s<18] = train_df['Age'].median()
train_df['Age'] = s
s = pd.Series(train_df['Age'])
s[s>120] = train_df['Age'].median()
train_df['Age'] = s
train_df['age_range'] = pd.cut(train_df['Age'], [0,20,30,65,100], labels=["0-20", "21-30", "31-65", "66-100"], include_lowest=True)

In [6]:
train_df['self_employed'] = train_df['self_employed'].replace([defaultString], 'No')
print(train_df['self_employed'].unique())

['No' 'Yes']


In [7]:
train_df['work_interfere'] = train_df['work_interfere'].replace([defaultString], 'Don\'t know' )
print(train_df['work_interfere'].unique())

['Often' 'Rarely' 'Never' 'Sometimes' "Don't know"]


In [8]:
train_df.to_csv('mental1.csv')

In [9]:
labelDict = {}
for feature in train_df:
    le = preprocessing.LabelEncoder()
    le.fit(train_df[feature])
    le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
    train_df[feature] = le.transform(train_df[feature])
    labelKey = 'label_' + feature
    labelValue = [*le_name_mapping]
    labelDict[labelKey] =labelValue
    
for key, value in labelDict.items():     
    print(key, value)
train_df.head()

label_Age [18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56, 57, 58, 60, 61, 62, 65, 72]
label_Gender ['female', 'male', 'trans']
label_self_employed ['No', 'Yes']
label_family_history ['No', 'Yes']
label_treatment ['No', 'Yes']
label_work_interfere ["Don't know", 'Never', 'Often', 'Rarely', 'Sometimes']
label_no_employees ['01-May', '100-500', '26-100', '500-1000', 'Jun-25', 'More than 1000']
label_remote_work ['No', 'Yes']
label_tech_company ['No', 'Yes']
label_benefits ["Don't know", 'No', 'Yes']
label_care_options ['No', 'Not sure', 'Yes']
label_wellness_program ["Don't know", 'No', 'Yes']
label_seek_help ["Don't know", 'No', 'Yes']
label_anonymity ["Don't know", 'No', 'Yes']
label_leave ["Don't know", 'Somewhat difficult', 'Somewhat easy', 'Very difficult', 'Very easy']
label_mental_health_consequence ['Maybe', 'No', 'Yes']
label_phys_health_consequence ['Maybe', 'No', 'Yes']
labe

,Age,Gender,self_employed,family_history,treatment,work_interfere,no_employees,remote_work,tech_company,benefits,...,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,Have you had a mental health disorder in the past?,Have you been diagnosed with a mental health condition by a medical professional?,age_range
0,19,0,0,0,1,2,4,0,1,2,...,1,1,2,1,0,2,0,2,1,2
1,26,1,0,0,0,3,5,0,0,0,...,1,0,0,1,1,0,0,2,1,2
2,14,1,0,0,0,3,4,0,1,1,...,1,2,2,2,2,1,0,0,0,2
3,13,1,0,1,1,2,2,0,1,1,...,2,1,0,0,0,1,1,2,1,2
4,13,1,0,0,0,1,1,1,1,2,...,1,1,2,2,2,0,0,2,1,2


In [10]:
total = train_df.isnull().sum().sort_values(ascending=False)
percent = (train_df.isnull().sum()/train_df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)
print(missing_data)

                                                    Total  Percent
age_range                                               0      0.0
Have you been diagnosed with a mental health co...      0      0.0
Gender                                                  0      0.0
self_employed                                           0      0.0
family_history                                          0      0.0
treatment                                               0      0.0
work_interfere                                          0      0.0
no_employees                                            0      0.0
remote_work                                             0      0.0
tech_company                                            0      0.0
benefits                                                0      0.0
care_options                                            0      0.0
wellness_program                                        0      0.0
seek_help                                               0     

In [11]:
train_df.to_csv('mental2.csv')

In [12]:
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.tree import DecisionTreeClassifier


from sklearn import metrics
from sklearn.metrics import accuracy_score, mean_squared_error, precision_recall_curve
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

In [13]:
scaler = MinMaxScaler()
train_df['Age'] = scaler.fit_transform(train_df[['Age']])
train_df.head()

,Age,Gender,self_employed,family_history,treatment,work_interfere,no_employees,remote_work,tech_company,benefits,...,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,Have you had a mental health disorder in the past?,Have you been diagnosed with a mental health condition by a medical professional?,age_range
0,0.431818,0,0,0,1,2,4,0,1,2,...,1,1,2,1,0,2,0,2,1,2
1,0.590909,1,0,0,0,3,5,0,0,0,...,1,0,0,1,1,0,0,2,1,2
2,0.318182,1,0,0,0,3,4,0,1,1,...,1,2,2,2,2,1,0,0,0,2
3,0.295455,1,0,1,1,2,2,0,1,1,...,2,1,0,0,0,1,1,2,1,2
4,0.295455,1,0,0,0,1,1,1,1,2,...,1,1,2,2,2,0,0,2,1,2


In [15]:
#train_df.to_csv('mental3.csv')

In [16]:
feature_cols = ['age_range', 'Gender', 'self_employed', 'family_history', 'work_interfere','remote_work','wellness_program', 'seek_help','mental_health_consequence','coworkers','Have you had a mental health disorder in the past?']
X = train_df[feature_cols]
y = train_df.treatment

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)


In [17]:
import pickle

In [18]:
def treeClassifier():
    
    # train a decision tree model on the training set
    tree = DecisionTreeClassifier(max_depth=3, min_samples_split=8, max_features=11, criterion='entropy', min_samples_leaf=7)
    tree.fit(X_train, y_train)
    
    # make class predictions for the testing set
    y_pred_class = tree.predict(X_test)
    pickle.dump(tree, open('model.pkl','wb'))
    model = pickle.load(open('model.pkl','rb'))
    print(model.predict([[1, 1, 0, 1, 0, 2, 1, 1, 0, 1, 0]]))
    print(model.predict([[2, 0, 1, 4, 1, 1, 1, 0, 1, 0, 2]]))

In [19]:
treeClassifier()

[0]
[0]
